In [7]:
"""
Backward Chaining Theorem Prover
Query: Prove "John Likes Peanuts"

Knowledge Base:
1. Apple and Vegetables are Food
2. Anything anyone eats and is not killed (alive) is food
3. Anil eats everything and is still alive
4. Harry eats everything that Anil eats
5. If food is eaten by someone, they like it
"""

from typing import List, Dict, Set, Optional, Tuple
from dataclasses import dataclass
import copy


@dataclass
class Predicate:
    """Represents a predicate with name and arguments"""
    name: str
    args: List[str]

    def __str__(self):
        return f"{self.name}({', '.join(self.args)})"

    def __hash__(self):
        return hash((self.name, tuple(self.args)))

    def __eq__(self, other):
        return self.name == other.name and self.args == other.args


@dataclass
class Rule:
    """Represents a rule with premises and conclusion"""
    id: str
    premises: List[Predicate]
    conclusion: Predicate
    description: str

    def __str__(self):
        premises_str = " ∧ ".join(str(p) for p in self.premises)
        return f"{self.id}: {premises_str} → {self.conclusion}"


class BackwardChaining:
    def __init__(self):
        self.facts: List[Predicate] = []
        self.rules: List[Rule] = []
        self.proof_steps: List[str] = []
        self.step_count = 0

    def add_fact(self, fact: Predicate):
        """Add a fact to the knowledge base"""
        self.facts.append(fact)

    def add_rule(self, rule: Rule):
        """Add a rule to the knowledge base"""
        self.rules.append(rule)

    def unify(self, pattern: Predicate, fact: Predicate,
              bindings: Dict[str, str] = None) -> Optional[Dict[str, str]]:
        """
        Unify a pattern with a fact, returning bindings if successful
        Variables start with '?'
        """
        if bindings is None:
            bindings = {}

        bindings = copy.copy(bindings)

        # Predicates must have the same name
        if pattern.name != fact.name:
            return None

        # Must have same number of arguments
        if len(pattern.args) != len(fact.args):
            return None

        # Try to unify each argument
        for p_arg, f_arg in zip(pattern.args, fact.args):
            if p_arg.startswith('?'):  # Variable
                if p_arg in bindings:
                    if bindings[p_arg] != f_arg:
                        return None
                else:
                    bindings[p_arg] = f_arg
            else:  # Constant
                if p_arg != f_arg:
                    return None

        return bindings

    def apply_bindings(self, predicate: Predicate,
                       bindings: Dict[str, str]) -> Predicate:
        """Apply variable bindings to a predicate"""
        new_args = [bindings.get(arg, arg) for arg in predicate.args]
        return Predicate(predicate.name, new_args)

    def log_step(self, depth: int, message: str):
        """Log a proof step with indentation"""
        indent = "  " * depth
        step_msg = f"{indent}{message}"
        self.proof_steps.append(step_msg)
        print(step_msg)

    def backward_chain(self, goal: Predicate, depth: int = 0,
                       visited: Set[str] = None) -> Tuple[bool, Dict[str, str]]:
        """
        Backward chaining algorithm to prove a goal
        Returns (proved, bindings)
        """
        if visited is None:
            visited = set()

        goal_str = str(goal)
        self.step_count += 1

        self.log_step(depth, f"[Step {self.step_count}] Goal: Prove {goal_str}")

        # Prevent infinite loops
        if goal_str in visited:
            self.log_step(depth, f"⚠ Cycle detected for {goal_str}")
            return False, {}

        visited.add(goal_str)

        # Check if goal is already a known fact
        for fact in self.facts:
            bindings = self.unify(goal, fact)
            if bindings is not None:
                self.log_step(depth, f"✓ Found fact: {fact}")
                return True, bindings

        # Try to prove using rules
        for rule in self.rules:
            bindings = self.unify(goal, rule.conclusion)

            if bindings is not None:
                self.log_step(depth, f"→ Trying rule {rule.id}: {rule.description}")

                # Try to prove all premises
                all_proved = True
                combined_bindings = copy.copy(bindings)

                for premise in rule.premises:
                    # Apply current bindings to premise
                    instantiated_premise = self.apply_bindings(premise, combined_bindings)
                    premise_str = str(instantiated_premise)

                    self.log_step(depth + 1, f"Subgoal: {premise_str}")

                    # Recursively prove the premise
                    proved, new_bindings = self.backward_chain(
                        instantiated_premise, depth + 2, copy.copy(visited)
                    )

                    if proved:
                        # Update bindings
                        combined_bindings.update(new_bindings)
                        self.log_step(depth + 1, f"✓ Subgoal proved: {premise_str}")
                    else:
                        self.log_step(depth + 1, f"✗ Subgoal failed: {premise_str}")
                        all_proved = False
                        break

                if all_proved:
                    self.log_step(depth, f"✓ Proved {goal_str} using rule {rule.id}")
                    return True, combined_bindings
                else:
                    self.log_step(depth, f"✗ Rule {rule.id} failed")

        self.log_step(depth, f"✗ Failed to prove {goal_str}")
        return False, {}

    def prove(self, query: Predicate):
        """Prove a query and display results"""
        print("=" * 80)
        print("BACKWARD CHAINING PROOF")
        print("=" * 80)
        print(f"\nQuery: {query}")
        print("\n" + "-" * 80)
        print("PROOF STEPS:")
        print("-" * 80 + "\n")

        self.proof_steps = []
        self.step_count = 0

        proved, bindings = self.backward_chain(query, 0)

        print("\n" + "=" * 80)
        print("RESULT:")
        print("=" * 80)

        if proved:
            print(f"✓ QUERY PROVED: {query} is TRUE")
            if bindings:
                print(f"\nVariable Bindings: {bindings}")
        else:
            print(f"✗ QUERY FAILED: Cannot prove {query}")

        print("=" * 80)

        return proved

    def display_knowledge_base(self):
        """Display the current knowledge base"""
        print("\n" + "=" * 80)
        print("KNOWLEDGE BASE")
        print("=" * 80)

        print("\nFACTS:")
        print("-" * 80)
        for i, fact in enumerate(self.facts, 1):
            print(f"{i}. {fact}")

        print("\nRULES:")
        print("-" * 80)
        for rule in self.rules:
            print(f"{rule.id}. {rule.description}")
            print(f"   {rule}")

        print("=" * 80 + "\n")


def main():
    """Main function to set up and run the backward chaining proof"""

    # Initialize the backward chaining system
    bc = BackwardChaining()

    # Add facts to knowledge base
    bc.add_fact(Predicate("Food", ["Apple"]))
    bc.add_fact(Predicate("Food", ["Vegetables"]))
    bc.add_fact(Predicate("Eats", ["Anil", "Peanuts"]))
    bc.add_fact(Predicate("Eats", ["Anil", "Apple"]))
    bc.add_fact(Predicate("Alive", ["Anil"]))

    # Add rules to knowledge base

    # Rule 1: If X eats Y and X is alive, then Y is food
    bc.add_rule(Rule(
        id="R1",
        premises=[
            Predicate("Eats", ["?x", "?y"]),
            Predicate("Alive", ["?x"])
        ],
        conclusion=Predicate("Food", ["?y"]),
        description="∀x,y: Eats(x,y) ∧ Alive(x) → Food(y)"
    ))

    # Rule 2: Harry eats everything that Anil eats
    bc.add_rule(Rule(
        id="R2",
        premises=[
            Predicate("Eats", ["Anil", "?x"])
        ],
        conclusion=Predicate("Eats", ["Harry", "?x"]),
        description="∀x: Eats(Anil, x) → Eats(Harry, x)"
    ))

    # Rule 3: John eats everything that Harry eats
    bc.add_rule(Rule(
        id="R3",
        premises=[
            Predicate("Eats", ["Harry", "?x"])
        ],
        conclusion=Predicate("Eats", ["John", "?x"]),
        description="∀x: Eats(Harry, x) → Eats(John, x)"
    ))

    # Rule 4: If Y is food and X eats Y, then X likes Y
    bc.add_rule(Rule(
        id="R4",
        premises=[
            Predicate("Food", ["?y"]),
            Predicate("Eats", ["?x", "?y"])
        ],
        conclusion=Predicate("Likes", ["?x", "?y"]),
        description="∀x,y: Food(y) ∧ Eats(x,y) → Likes(x,y)"
    ))

    # Display knowledge base
    bc.display_knowledge_base()

    # Query: Prove "John Likes Peanuts"
    query = Predicate("Likes", ["John", "Peanuts"])

    # Run the proof
    bc.prove(query)

    print("\n" + "=" * 80)
    print("PROOF TREE VISUALIZATION")
    print("=" * 80)
    print("""
Likes(John, Peanuts) [R4]
├── Food(Peanuts) [R1]
│   ├── Eats(Anil, Peanuts) [FACT] ✓
│   └── Alive(Anil) [FACT] ✓
└── Eats(John, Peanuts) [R3]
    └── Eats(Harry, Peanuts) [R2]
        └── Eats(Anil, Peanuts) [FACT] ✓

PROOF CHAIN:
1. Eats(Anil, Peanuts) - Given as fact
2. Alive(Anil) - Given as fact
3. Food(Peanuts) - From R1: Eats(Anil, Peanuts) ∧ Alive(Anil)
4. Eats(Harry, Peanuts) - From R2: Eats(Anil, Peanuts)
5. Eats(John, Peanuts) - From R3: Eats(Harry, Peanuts)
6. Likes(John, Peanuts) - From R4: Food(Peanuts) ∧ Eats(John, Peanuts) ✓
""")
    print("=" * 80)


if __name__ == "__main__":
    main()


KNOWLEDGE BASE

FACTS:
--------------------------------------------------------------------------------
1. Food(Apple)
2. Food(Vegetables)
3. Eats(Anil, Peanuts)
4. Eats(Anil, Apple)
5. Alive(Anil)

RULES:
--------------------------------------------------------------------------------
R1. ∀x,y: Eats(x,y) ∧ Alive(x) → Food(y)
   R1: Eats(?x, ?y) ∧ Alive(?x) → Food(?y)
R2. ∀x: Eats(Anil, x) → Eats(Harry, x)
   R2: Eats(Anil, ?x) → Eats(Harry, ?x)
R3. ∀x: Eats(Harry, x) → Eats(John, x)
   R3: Eats(Harry, ?x) → Eats(John, ?x)
R4. ∀x,y: Food(y) ∧ Eats(x,y) → Likes(x,y)
   R4: Food(?y) ∧ Eats(?x, ?y) → Likes(?x, ?y)

BACKWARD CHAINING PROOF

Query: Likes(John, Peanuts)

--------------------------------------------------------------------------------
PROOF STEPS:
--------------------------------------------------------------------------------

[Step 1] Goal: Prove Likes(John, Peanuts)
✗ Failed to prove Likes(John, Peanuts)

RESULT:
✗ QUERY FAILED: Cannot prove Likes(John, Peanuts)

PROOF TR